In [ ]:
## Insert Data Cleaning Part from Juanjo

#-----------FOR THE BOIS----------------------

#--------LIBRARIES FOR THE BOIS (THE ONES I USED, THE USUAL ONES)-----------

#COOL LIBRARIES
import pandas as pd
import numpy  as np
import os




#----------FUNCTIONS FOR THE BOIS-----------

#CREATE DATAFRAMES FUNCTION
#Creates 3 super cool dataframes from the CSVs with the data types set from the start.
def dataFrameCreate():
    global sales_phases_funnel_df, zipcode_df, meteo_df

    # SET PATHS OF 3 COOL CSVs
    FILENAME_sales_phases_funnel_df = os.path.join(os.getcwd(), r'data/sale_phases_funnel.csv')
    FILENAME_zipcode_df = os.path.join(os.getcwd(), r'data/zipcode_eae.csv')
    FILENAME_meteo_df = os.path.join(os.getcwd(), r'data/meteo_eae.csv')


    #SALES FUNNEL DATAFRAME

    #Dictionary with data types
    SALES_TYPES = {'LEAD_ID':'str','FINANCING_TYPE':'str',
                    'CURRENT_PHASE':'str','PHASE_PRE_KO':'str',
                    'IS_MODIFIED':'bool','ZIPCODE':'str', 
                    'VISITING_COMPANY': 'str', 'KO_REASON': 'str', 
                    'INSTALLATION_PEAK_POWER_KW': 'float64', 
                    'INSTALLATION_PRICE': 'float', 
                    'N_PANELS': 'int', 'CUSOMER_TYPE': 'str' }

    # Reading CSV to create dataframe with datatypes implemented from dictionary and additional date time datatypes.
    sales_phases_funnel_df = pd.read_csv(
        FILENAME_sales_phases_funnel_df, 
        delimiter=';', 
        dtype=SALES_TYPES,
        parse_dates=['OFFER_SENT_DATE', 'CONTRACT_1_DISPATCH_DATE', 
                    'CONTRACT_2_DISPATCH_DATE', 
                    'CONTRACT_1_SIGNATURE_DATE', 
                    'CONTRACT_2_SIGNATURE_DATE',
                    'VISIT_DATE',
                    'TECHNICAL_REVIEW_DATE',
                    'PROJECT_VALIDATION_DATE',
                    'SALE_DISMISSAL_DATE',
                    'KO_DATE'],
                    
        dayfirst=True  # This replaces the dayfirst=True in your to_datetime call
    )

    print('sales_phases_funnel_df created')



    #ZIPCODE DATAFRAME

    #Dictionary with data types
    ZIPCODE_TYPES = {'ZIPCODE':'str','ZC_LATITUDE':'float64',
                    'ZC_LONGITUDE':'float64','AUTONOMOUS_COMMUNITY':'str',
                    'AUTONOMOUS_COMMUNITY_NK':'str','PROVINCE':'str'}


    # Reading CSV to create dataframe with datatypes implemented from dictionary
    zipcode_df = pd.read_csv(FILENAME_zipcode_df, delimiter=',', dtype=ZIPCODE_TYPES)


    print('zipcodedf created')





    #METEO DATAFRAME

    #Dictionary with data types
    METEO_TYPES = {'temperature': 'float', 'relative_humidity': 'float', 
                'precipitation_rate': 'float', 'wind_speed': 'float', 
                'zipcode': 'str' 
    }

    # Reading CSV to create dataframe with datatypes implemented from dictionary and 
    # additional date time datatype formatted to match the ones from the sales dataframe.
    meteo_df = pd.read_csv(FILENAME_meteo_df, delimiter=';',
        dtype=METEO_TYPES, parse_dates=['date'],  # Replace with actual column name
        date_format='%Y/%m/%d %H:%M:%S.%f'  # This matches your input format
    )

    print('meteo_df created')
    



dataFrameCreate()




#--GLOBAL CLEANING FUNCTION--


#DROPPING DUPLICATES FOR ALL DATAFRAMES

#creating the drop duplicate function
def dropDupli():
    global sales_phases_funnel_df, zipcode_df, meteo_df
    print('There are ', sales_phases_funnel_df.duplicated().sum(), ' duplicate rows in sales_funnel_df before duplicate cleaning') 
    print('There are ', zipcode_df.duplicated().sum(), ' duplicate rows in zipcode_df before duplicate cleaning')
    print('There are ', meteo_df.duplicated().sum(), ' duplicate rows in meteo_df before duplicate cleaning') 
    sales_phases_funnel_df.drop_duplicates(inplace=True)
    zipcode_df.drop_duplicates(inplace=True)
    meteo_df.drop_duplicates(inplace=True)
    print('There are ', sales_phases_funnel_df.duplicated().sum(), ' duplicate rows in sales_funnel_df after duplicate cleaning') 
    print('There are ', zipcode_df.duplicated().sum(), ' duplicate rows in zipcode_df after duplicate cleaning')
    print('There are ', meteo_df.duplicated().sum(), ' duplicate rows in meteo_df after duplicate cleaning')

dropDupli()





# --SALES FUNNEL DATAFRAME CLEANING FUCTIONS--


#DELETE UNUSABLE LEADS FUNCTION 

# Drop rows where KO_REASON is "Unreachable"
def delete_unreachable_leads():
    global sales_phases_funnel_df
    sales_phases_funnel_df = sales_phases_funnel_df[~((sales_phases_funnel_df['CURRENT_PHASE'] == 'KO') & (sales_phases_funnel_df['KO_REASON'] == 'Unreachable'))]
    # Reset the index of the updated DataFrame
    sales_phases_funnel_df.reset_index(drop=True, inplace=True)

# Verify the changes
#print(sales_phases_funnel_df.isnull().sum())
#print(sales_phases_funnel_df.head())

delete_unreachable_leads()




# REMOVE OUTLIERS FUNCTION


def delete_outliers():
    global sales_phases_funnel_df, outliers_df
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = sales_phases_funnel_df['INSTALLATION_PRICE'].quantile(0.25)
    Q3 = sales_phases_funnel_df['INSTALLATION_PRICE'].quantile(0.75)

    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1

    # Identify outliers inside a new Data Frame
    outliers_df = sales_phases_funnel_df[(sales_phases_funnel_df['INSTALLATION_PRICE'] < (Q1 - 1.5 * IQR)) | 
                                     (sales_phases_funnel_df['INSTALLATION_PRICE'] > (Q3 + 1.5 * IQR))]

    # Print the number of outliers
    print('outliers_df created')
    print(f'Number of outliers: {len(outliers_df)}')
    
    # Update sales_phases_funnel_df to exclude the outliers
    sales_phases_funnel_df = sales_phases_funnel_df[~((sales_phases_funnel_df['INSTALLATION_PRICE'] < (Q1 - 1.5 * IQR)) | 
                                                  (sales_phases_funnel_df['INSTALLATION_PRICE'] > (Q3 + 1.5 * IQR)))]
    print('outliers removed from sales_phases_funnel_df')
    


delete_outliers()






#-------------------------DATAFRAMES FOR THE BOIS-------


#This is the final list of dataframes
list_of_dfs = [sales_phases_funnel_df, zipcode_df, meteo_df, outliers_df]

In [ ]:
## -------Transformations---------- ##

# Rename Data Frames

sales_fact_df=sales_phases_funnel_df
zipcode_dim_df=zipcode_df
weather_dim_df=meteo_df

# All column names in lower case letters

sales_fact_df.columns= sales_fact_df.columns.str.lower()
zipcode_dim_df.columns= zipcode_dim_df.columns.str.lower()
weather_dim_df.columns= weather_dim_df.columns.str.lower()



In [ ]:
# Create zipcode_id in zipcode_dim_df

zipcode_dim_df.insert(0,"zipcode_id",range(1, len(zipcode_dim_df) + 1))
zipcode_dim_df["zipcode_id"] = zipcode_dim_df["zipcode_id"].astype("int32")
zipcode_df

In [ ]:
weather_dim_df

In [ ]:
# Merge zipcode_dim and weather_dim

weather_dim_df = pd.merge(weather_dim_df,zipcode_dim_df,on= "zipcode", how="left")
weather_dim_df

In [ ]:
# Merge zipcode_dim and sales_fact

sales_fact_df = pd.merge(sales_fact_df, zipcode_dim_df, on="zipcode", how="left")
sales_fact_df

In [ ]:
# Add weather_id in weather_dim

weather_dim_df.insert(0,"weather_id",range(1, len(weather_dim_df) + 1))

In [ ]:
# Add sales_id to sales_fact

sales_fact_df.insert(0,"sales_id",range(1, len(sales_fact_df) + 1))

In [ ]:
# Add calculated column to sales_fact_df

sales_fact_df.insert(16,"most_recent_contract_signature",sales_fact_df[["contract_1_signature_date", "contract_2_signature_date"]].max(axis=1))

In [ ]:
# Final Arrangement of columns for sales_fact_df

FINAL_COLS_SALES = ["sales_id","zipcode_id","lead_id","financing_type","current_phase","phase_pre_ko",
              "is_modified","offer_sent_date","contract_1_dispatch_date","contract_2_dispatch_date","contract_1_signature_date",
              "contract_2_signature_date","most_recent_contract_signature","visit_date","technical_review_date",
              "project_validation_date","sale_dismissal_date","ko_date","visiting_company","ko_reason",
              "installation_peak_power_kw","installation_price","n_panels","cusomer_type"]

In [ ]:
# Change date to year, rename columns in and final arrangement of columns in weather_dim_df

weather_dim_df['date'] = weather_dim_df['date'].dt.year

weather_dim_df= weather_dim_df.rename(columns={"date":"year",'temperature': 'avg_temperature', 'relative_humidity': 'avg_relative_humidity',
                                               "precipitation_rate":"avg_precipitation_rate","wind_speed":"avg_wind_speed"})



FINAL_COLS_WEATHER=["weather_id","zipcode_id","year","avg_temperature","avg_relative_humidity","avg_precipitation_rate",
                    "avg_wind_speed"]



In [ ]:
zipcode_dim_df

In [ ]:
weather_dim_df=weather_dim_df[FINAL_COLS_WEATHER]

weather_dim_df

In [ ]:
sales_fact_df=sales_fact_df[FINAL_COLS_SALES]
sales_fact_df

In [ ]:
# To Do:

# Group weather table by zipcode id 
# Create zipcode id in zipcode_dim
# Merge zipcode and weather 
# merge zipcode and fact
# add weather id in weather_dim done 
# add sales id to sales done
# columns

